# **Welcome to the notebook**

### Task 1 - Set up project environment

Installing the needed modules

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install openai==1.16.2 python-dotenv

Importing the needed modules and setup the OpenAI API

In [ ]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import os
import openai
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv(dotenv_path='/content/apikey.env.txt')

# Retrieve API key from environment variables
API_KEY = os.getenv("APIKEY")

# Set the API key globally
openai.api_key = API_KEY


In [ ]:
import openai
print(openai.__version__)

1.16.2


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

### Task 2 - Prepare the Kowledge Base

Loading the products dataset

In [ ]:
import pandas as pd
products_data = pd.read_csv('/content/products_dataset.csv')
products_data.head()


,product_id,title,description
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat..."
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...


Combining product `title` and `description`

In [ ]:
products_data['item'] = products_data['title'] + ' ' + products_data['description']
products_data.head()

,product_id,title,description,item
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat...",Men's 3X Large Carbon Heather Cotton/Polyester...
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...,Large Tapestry Bolster Bed Polyester cover res...
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,16-Gauge-Sinks Vessel Sink in White with Fauce...
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...


Creating the text embedding vectors

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import torch

# Load Sentence Transformer model
model_id = "BAAI/bge-base-en"
model = SentenceTransformer(model_id)

# Generate embeddings
products_data['embedding_vector'] = products_data['item'].apply(lambda x: model.encode(x))

# Display the updated dataframe
print(products_data.head())

  product_id                                              title  \
0         P0  Men's 3X Large Carbon Heather Cotton/Polyester...   
1         P1  Turmode 30 ft. RP TNC Female to RP TNC Male Ad...   
2         P2                         Large Tapestry Bolster Bed   
3         P3    16-Gauge-Sinks Vessel Sink in White with Faucet   
4         P4  Men's Crazy Horse 9'' Logger Boot - Steel Toe ...   

                                         description  \
0  This heavyweight, water-repellent hooded sweat...   
1  If you need more length between your existing ...   
2  Polyester cover resembling rich Italian tapest...   
3  It features a rectangle shape. This vessel set...   
4  This 9 in. black full grain leather logger boo...   

                                                item  \
0  Men's 3X Large Carbon Heather Cotton/Polyester...   
1  Turmode 30 ft. RP TNC Female to RP TNC Male Ad...   
2  Large Tapestry Bolster Bed Polyester cover res...   
3  16-Gauge-Sinks Vessel Sink in Whi

### Task 3 - Retrieve relevant items based on user prompts

In [ ]:
user_prompt = "I'm remodeling my bathroom and need a stylish white sink with a faucet included."

Let's find the embedding vector of the user prompt.

In [ ]:
prompt_embedding = model.encode(user_prompt)

Find similar products to the user prompt

In [ ]:
similarities = cosine_similarity([prompt_embedding], list(products_data['embedding_vector']))[0]


products_data['similarity'] = similarities


top_10 = products_data.nlargest(10, 'similarity')


### Task 4 - Craft the context-rich prompt

Getting the top-10 similiar items

In [ ]:
print(top_10[['item', 'similarity']])

                                                   item  similarity
347   Modern in White Ceramic Rectangular Vessel Sin...    0.843499
588   16-Gauge-Sinks 23 in. W x 18 in. D Bath Vanity...    0.839030
569   16.5 in. W x 16.5 in. D Ceramic Vanity Top in ...    0.836593
3     16-Gauge-Sinks Vessel Sink in White with Fauce...    0.833978
759   Deborah 72 in. W x 22 in. D x 35 in. H Double ...    0.827176
1352  31 in. W x 22 in. D Engineered Marble Vanity T...    0.825408
79    24.38 in. x 14 in. x 6 in. White Above Counter...    0.823603
940   Grail China Vessel Sink in White with Overflow...    0.823451
1021  16-Gauge-Sinks Vessel Sink in White It feature...    0.823354
1180  Traditional in White Ceramic Oval Vessel Sink ...    0.822698


Let's combine the information about all the related products.

In [ ]:
combined_info = '\n \n '.join(top_10['item'])
print(combined_info)

Modern in White Ceramic Rectangular Vessel Sink with Faucet Included with Overflow Drain Included It features a rectangle shape. This vessel set is designed to be installed as a above counter vessel set. It is constructed with ceramic. This vessel set comes with an enamel glaze finish in White color. It is designed for a 1 hole faucet.
 
 16-Gauge-Sinks 23 in. W x 18 in. D Bath Vanity Cabinet Only in White This 16-Gauge-Sinks modern vanity base belongs to the exquisite Bow design series. It features a rectangle shape. This 16-Gauge-Sinks vanity base is designed to be installed as a floor mount vanity base. It is constructed with birch wood-veneer. This 16-Gauge-Sinks vanity base comes with a lacquer-paint finish in White color.
 
 16.5 in. W x 16.5 in. D Ceramic Vanity Top in White with White Rectangular Single Sink It features a square shape. This ceramic top set is designed to be installed as a drop in ceramic top set. It is constructed with ceramic. This ceramic top set comes with a

Now let's craft our prompt

In [ ]:
prompt = f"""
Imagine your role as an expert customer service agent. We have the following products:
{combined_info}

Based on the user prompt and the products listed above, recommend 3 products to the user.
The user prompt is: {user_prompt}

"""

print(prompt)


Imagine your role as an expert customer service agent. We have the following products:
Modern in White Ceramic Rectangular Vessel Sink with Faucet Included with Overflow Drain Included It features a rectangle shape. This vessel set is designed to be installed as a above counter vessel set. It is constructed with ceramic. This vessel set comes with an enamel glaze finish in White color. It is designed for a 1 hole faucet.
 
 16-Gauge-Sinks 23 in. W x 18 in. D Bath Vanity Cabinet Only in White This 16-Gauge-Sinks modern vanity base belongs to the exquisite Bow design series. It features a rectangle shape. This 16-Gauge-Sinks vanity base is designed to be installed as a floor mount vanity base. It is constructed with birch wood-veneer. This 16-Gauge-Sinks vanity base comes with a lacquer-paint finish in White color.
 
 16.5 in. W x 16.5 in. D Ceramic Vanity Top in White with White Rectangular Single Sink It features a square shape. This ceramic top set is designed to be installed as a dr

### Task 5 - Prompt the LLM to generate product recommendation

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, pipeline

In [ ]:
!pip install transformers torch bitsandbytes accelerate

In [ ]:
from google.colab import userdata
llama_key = userdata.get('HF_TOKEN')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [ ]:
!pip install transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0.dev0
    Uninstalling transformers-4.50.0.dev0:
      Successfully uninstalled transformers-4.50.0.dev0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.


In [ ]:
!pip install git+https://github.com/PanQiWei/AutoGPTQ.git@e9008a3e6b3a6965a9a5a9a4e9cd84e3a63c52e0

  Cloning https://github.com/PanQiWei/AutoGPTQ.git (to revision e9008a3e6b3a6965a9a5a9a4e9cd84e3a63c52e0) to /tmp/pip-req-build-y0lv1_pk
  Running command git clone --filter=blob:none --quiet https://github.com/PanQiWei/AutoGPTQ.git /tmp/pip-req-build-y0lv1_pk
  Running command git rev-parse -q --verify 'sha^e9008a3e6b3a6965a9a5a9a4e9cd84e3a63c52e0'
  Running command git fetch -q https://github.com/PanQiWei/AutoGPTQ.git e9008a3e6b3a6965a9a5a9a4e9cd84e3a63c52e0
  fatal: remote error: upload-pack: not our ref e9008a3e6b3a6965a9a5a9a4e9cd84e3a63c52e0
  error: subprocess-exited-with-error
  
  × git fetch -q https://github.com/PanQiWei/AutoGPTQ.git e9008a3e6b3a6965a9a5a9a4e9cd84e3a63c52e0 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git fetch -q https://github.com/PanQiWei/AutoGPTQ.git e9008a3e6b3a6965a9a5a9a4e9cd84e3a63c52e0 did not r

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_id = "meta-llama/Meta-Llama-3-8B"

   # Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

   # Define quantization configuration
quantize_config = BaseQuantizeConfig(
       bits=4,  # Adjust the number of bits for quantization as needed
       group_size=128,  # Adjust group size as needed
   )

   # Load quantized model (GPTQ) on CPU
model = AutoGPTQForCausalLM.from_quantized(
       model_id,
       quantize_config=quantize_config,  # Pass the config explicitly
       device="cpu",  # Change to "cuda" if you later enable GPU
       use_safetensors=True,
       trust_remote_code=True,  # Add this if loading from Hugging Face Hub
   )

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads a

In [ ]:
text_generator = pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=512,
)

Device set to use cpu
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM

In [ ]:
def generate_response(prompt):
    response = text_generator(prompt)
    gen_text = response[0]['generated_text']
    return gen_text

print("User:", user_prompt)
print("Agent:", generate_response(prompt))

User: I'm remodeling my bathroom and need a stylish white sink with a faucet included.


RuntimeError: Unexpected floating ScalarType in at::autocast::prioritize

In [ ]:


# Use a valid open-source LLM
llm_pipeline = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1", device=0)

response = llm_pipeline(user_prompt, max_length=1000)

print("User:", user_prompt)
print("Agent:", response[0]['generated_text'])
